In [1]:
from statsmodels.tsa import holtwinters as hw
import statsmodels.api as sm
import pandas as pd
import numpy as np
import plotly.express as px
import plotly
import plotly.graph_objs as go

FREQ = 'QS'

## Данные

In [99]:


# Read in data and convert index to a datetime
period = 'Monthly'
df = pd.read_csv(f'data_seminars/m4_sample/{period}_sample.csv',
                 index_col=0)

# Ряды не размечены по датам, поэтому для удобства возьмём произвольную точку старта
df.index = pd.date_range(start='01-01-2000',
                         periods=df.shape[0],
                         freq='M')
a = np.random.choice(df.columns)
print(a)
ts = df[a]


fig = px.line(x=ts.index, y=ts.values);
fig.update_layout(title='Some forecasting model',
                   xaxis_title='Time',
                   yaxis_title='Value');
fig.update_xaxes(rangeslider_visible=True);
fig.show()


# Функция, которая рисует прогнозы
def forplot(mul=False):
    fig = go.Figure()

    fig.add_trace(go.Scatter( x=ts.index,
                              y=ts.values)
                 )
    
    if mul:
         fig.add_trace(go.Scatter(x=forecs.index,
                             y=forecs.values))
    else:
        fig.add_trace(go.Scatter(x=forecs.index,
                                 y=forecs['mean']))

    fig.update_xaxes(rangeslider_visible=True);
    fig.show()



col_1342


## Простое экспоненциальное сглаживание, ETS(A,N,N)

In [100]:
exp = sm.tsa.statespace.ExponentialSmoothing(ts)
res = exp.fit()
print(res.summary())


forecs = res.get_forecast(50).summary_frame()

forplot()

                        Exponential Smoothing Results                         
Dep. Variable:               col_1342   No. Observations:                   77
Model:                   ETS(A, N, N)   Log Likelihood                -340.371
Date:                Tue, 29 Sep 2020   AIC                            686.742
Time:                        01:18:17   BIC                            693.773
Sample:                    01-31-2000   HQIC                           689.554
                         - 05-31-2006   Scale                          404.610
Covariance Type:                  opg                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
smoothing_level     0.9999      0.133      7.510      0.000       0.739       1.261
initial_level    1842.7697   2.47e+05      0.007      0.994   -4.83e+05    4.87e+05
Ljung-Box (Q):                  

## Экспоненциальное сглаживание с сезонностью, ETS(A,N,A)

In [101]:
mod1 = exp = sm.tsa.statespace.ExponentialSmoothing(ts, seasonal=12)
res = exp.fit(maxiter=500)
print(res.summary())


forecs = res.get_forecast(50).summary_frame()

forplot()


                        Exponential Smoothing Results                         
Dep. Variable:               col_1342   No. Observations:                   77
Model:                   ETS(A, N, A)   Log Likelihood                -255.717
Date:                Tue, 29 Sep 2020   AIC                            541.434
Time:                        01:18:23   BIC                            576.591
Sample:                    01-31-2000   HQIC                           555.497
                         - 05-31-2006   Scale                           44.886
Covariance Type:                  opg                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
smoothing_level         0.9999      0.133      7.491      0.000       0.738       1.262
smoothing_seasonal      0.0001      0.125      0.001      0.999      -0.245       0.245
initial_level   

## Линейная модель Хольта-Винтерса, ETS(A,A,N)

In [107]:
mod1 = exp = sm.tsa.statespace.ExponentialSmoothing(ts,
                                                    trend=True, damped_trend=True)
res = exp.fit()
print(res.summary())


forecs = res.get_forecast(50).summary_frame()

forplot()


                        Exponential Smoothing Results                         
Dep. Variable:               col_1342   No. Observations:                   77
Model:                  ETS(A, Ad, N)   Log Likelihood                -340.153
Date:                Tue, 29 Sep 2020   AIC                            692.306
Time:                        10:44:45   BIC                            706.369
Sample:                    01-31-2000   HQIC                           697.931
                         - 05-31-2006   Scale                          402.329
Covariance Type:                  opg                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
smoothing_level     0.9999      0.147      6.818      0.000       0.712       1.287
smoothing_trend     0.0001      0.082      0.001      0.999      -0.160       0.160
damping_trend       0.9800      

## Линейная модель Хольта-Винтерса с сезонностью, ETS(A,A,A)

In [103]:
mod1 = exp = sm.tsa.statespace.ExponentialSmoothing(ts,
                                                    trend=True,
                                                    damped_trend=True,
                                                    seasonal=12)
res = exp.fit(maxiter=1000)
print(res.summary())


forecs = res.get_forecast(50).summary_frame()

forplot()





                        Exponential Smoothing Results                         
Dep. Variable:               col_1342   No. Observations:                   77
Model:                  ETS(A, Ad, A)   Log Likelihood                -255.338
Date:                Tue, 29 Sep 2020   AIC                            546.676
Time:                        01:18:33   BIC                            588.864
Sample:                    01-31-2000   HQIC                           563.551
                         - 05-31-2006   Scale                           44.446
Covariance Type:                  opg                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
smoothing_level         0.9999      0.141      7.115      0.000       0.724       1.275
smoothing_trend         0.0388      0.047      0.825      0.409      -0.053       0.131
smoothing_season

## Альтернативная реализация с мультипликативными трендом и сезонностью

In [109]:
mod1 = exp = sm.tsa.ExponentialSmoothing(ts,
                                        trend='mul',
                                        damped=True,
                                        seasonal='add',
                                        seasonal_periods=12)
res = exp.fit()
print(res.summary())


forecs = res.forecast(50)

forplot(mul=True)


# hw.ExponentialSmoothing()


                       ExponentialSmoothing Model Results                       
Dep. Variable:                    endog   No. Observations:                   77
Model:             ExponentialSmoothing   SSE                          75570.583
Optimized:                         True   AIC                            564.454
Trend:                   Multiplicative   BIC                            604.299
Seasonal:                      Additive   AICC                           577.788
Seasonal Periods:                    12   Date:                 Tue, 29 Sep 2020
Box-Cox:                          False   Time:                         10:46:46
Box-Cox Coeff.:                    None                                         
                          coeff                 code              optimized      
---------------------------------------------------------------------------------
smoothing_level               0.7894737                alpha                 True
smoothing_slope          

C:\Users\The_sun\Anaconda3\envs\ts_course\lib\site-packages\statsmodels\tsa\holtwinters.py:725: RuntimeWarning:

invalid value encountered in less_equal

C:\Users\The_sun\Anaconda3\envs\ts_course\lib\site-packages\statsmodels\tsa\holtwinters.py:731: RuntimeWarning:

invalid value encountered in greater_equal

